In [14]:
import random
import torch

class SeatingArrangement:
    def __init__(self, n_people=24, rows=6, cols=4, constraint_matrix=None):
        self.n_people = n_people
        self.rows = rows
        self.cols = cols
        self.constraint_matrix = constraint_matrix

    def generate_seating_order(self):
        person_ids = list(range(self.n_people))
        random.shuffle(person_ids)

        seating_order = torch.zeros(self.rows, self.cols)
        for i in range(self.rows):
            for j in range(self.cols):
                seating_order[i, j] = person_ids[i * self.cols + j]

        return seating_order.float()

    def analyze_seating_order_conflicts(self, seating_order):
        conflicts = torch.zeros_like(seating_order)
        
        for i in range(self.rows):
            for j in range(self.cols):
                person = int(seating_order[i, j])
                if j > 0:
                    conflicts[i, j] += self.constraint_matrix[person, int(seating_order[i, j - 1])]
                if j < self.cols - 1:
                    conflicts[i, j] += self.constraint_matrix[person, int(seating_order[i, j + 1])]
                if i > 0:
                    conflicts[i, j] += self.constraint_matrix[person, int(seating_order[i - 1, j])]
                if i < self.rows - 1:
                    conflicts[i, j] += self.constraint_matrix[person, int(seating_order[i + 1, j])]

        return conflicts

    def find_incompatible_pairs(self, conflicts):
        incompatible_pairs = []
        for i in range(self.rows):
            for j in range(self.cols):
                if conflicts[i, j] != 0:
                    incompatible_pairs.append((i, j))
        return incompatible_pairs

    def get_conflicted_seats(self, seating_order, incompatible_pairs):
        conflicted_seats = [int(seating_order[i, j]) for i, j in incompatible_pairs]
        return conflicted_seats

    def swap_conflict_seats(self, incompatible_pairs, conflicted_seats, seating_order):
        for (i, j), value in zip(incompatible_pairs, conflicted_seats):
            seating_order[i, j] = value
        return seating_order

    def update_conflict_new_order(self, seating_order):
        return self.analyze_seating_order_conflicts(seating_order)


# Set random seed for reproducibility
random.seed(42)

# Generate random names
first_names = ["Ali", "Zahra", "Reza", "Sara", "Mohammad", "Fatemeh", "Hossein", "Maryam", "Mehdi", "Narges", "Hamed", "Roya"]
last_names = ["Ahmadi", "Hosseini", "Karimi", "Rahimi", "Hashemi", "Ebrahimi", "Moradi", "Mohammadi", "Rostami", "Fazeli", "Hosseinzadeh", "Niknam"]

random_names = [random.choice(first_names) + " " + random.choice(last_names) for _ in range(24)]

# Display the generated names
for name in random_names:
    print(name)

# Generate a constraint matrix for 24 people
constraint_matrix = torch.zeros((24, 24), dtype=torch.int)

# Randomly select 40 pairs of individuals who should not sit next to each other
constraints = set()
while len(constraints) < 40:
    i, j = random.sample(range(24), 2)
    if (i, j) not in constraints and (j, i) not in constraints:
        constraints.add((i, j))
        constraint_matrix[i, j] = 1
        constraint_matrix[j, i] = 1

# Display the constraint pairs
for i, j in constraints:
    print(f"{random_names[i]} should not sit next to {random_names[j]}")

print("\nConstraint Matrix:")
print(constraint_matrix)

import torch.nn as nn
import torch.optim as optim

class SeatingModel(nn.Module):
    def __init__(self):
        super(SeatingModel, self).__init__()
        self.fc1 = nn.Linear(24, 128)
        self.fc2 = nn.Linear(128, 24)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Instantiate the model, loss function, and optimizer
model = SeatingModel()
optimizer = optim.Adam(model.parameters())

def calculate_loss(output, constraint_matrix):
    probabilities = torch.softmax(output, dim=1).reshape(-1)
    loss = 0
    for i in range(23):
        for j in range(i + 1, 24):
            if constraint_matrix[i, j] == 1:
                loss += probabilities[i] * probabilities[j]
    return loss

# Training loop
rows, cols = 6, 4
seating_arrangement = SeatingArrangement(n_people=24, rows=6, cols=4, constraint_matrix=constraint_matrix)
seating_order = seating_arrangement.generate_seating_order()
seating_order_conflicts = seating_arrangement.analyze_seating_order_conflicts(seating_order)
print('\nseating_order:', seating_order)
print('seating_order_conflicts:', seating_order_conflicts)

# Number of epochs
n_epochs = 100
loss_values = []

for epoch in range(n_epochs):
    if epoch != 0:
        incompatible_pairs = seating_arrangement.find_incompatible_pairs(seating_order_conflicts)
        print('find incompatible pairs using constraint matrix:', incompatible_pairs)
        conflicted_seats = seating_arrangement.get_conflicted_seats(seating_order, incompatible_pairs)
        print('find conflicted seat numbers:', conflicted_seats)

        if len(conflicted_seats) == 1:
            break

        random.shuffle(conflicted_seats)
        new_seating_order = seating_arrangement.swap_conflict_seats(incompatible_pairs, conflicted_seats, seating_order)
        print('new generated seating order:', new_seating_order)
        updated_conflict = seating_arrangement.update_conflict_new_order(seating_order)
        print('updated conflict seating order:', updated_conflict)

        if len(conflicted_seats) == 2 and torch.sum(updated_conflict == 1) == 2:
            if loss_value < 1.000000e-06:
                break

        seating_order = new_seating_order
        seating_order_conflicts = updated_conflict

    # Predict seating probabilities
    output = model(torch.FloatTensor(seating_order_conflicts).view(-1, rows * cols))

    # Compute loss
    loss_value = calculate_loss(output, constraint_matrix)
    loss_values.append(loss_value.item())

    # Zero gradients, perform backward pass, and update weights
    optimizer.zero_grad()
    loss_value.backward()
    optimizer.step()

    # Display loss
    print(f"Epoch {epoch + 1}: loss = {loss_value.item():.7f}")

# Display all loss values
print("\nAll loss values:")
for epoch in range(1, n_epochs + 1):
    if epoch <= len(loss_values):
        print(f"Epoch {epoch}: loss = {loss_values[epoch - 1]:.7f}")
    else:
        print(f"Epoch {epoch}: loss = (not available)")


Hamed Hosseini
Ali Niknam
Mohammad Rahimi
Sara Karimi
Roya Hosseini
Hamed Niknam
Mehdi Hosseini
Narges Moradi
Ali Ahmadi
Zahra Rahimi
Sara Rostami
Narges Ahmadi
Mehdi Rahimi
Roya Hosseinzadeh
Roya Rostami
Hossein Rahimi
Maryam Fazeli
Mohammad Ahmadi
Reza Niknam
Hossein Ebrahimi
Mohammad Karimi
Sara Ebrahimi
Zahra Hosseini
Hossein Hosseini
Sara Ebrahimi should not sit next to Sara Rostami
Hossein Hosseini should not sit next to Narges Moradi
Roya Rostami should not sit next to Roya Hosseini
Mohammad Ahmadi should not sit next to Sara Karimi
Mohammad Ahmadi should not sit next to Zahra Rahimi
Zahra Rahimi should not sit next to Mohammad Rahimi
Sara Rostami should not sit next to Mehdi Hosseini
Narges Ahmadi should not sit next to Hamed Niknam
Narges Moradi should not sit next to Roya Hosseini
Reza Niknam should not sit next to Zahra Hosseini
Mohammad Karimi should not sit next to Hossein Ebrahimi
Hossein Hosseini should not sit next to Reza Niknam
Mohammad Karimi should not sit next to Z